In [1]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score
from scipy import spatial

In [2]:
cleaned_data = pd.read_csv('cleaned.csv')
cleaned_data = cleaned_data.dropna()


In [3]:
cleaned_data

,name_1,name_2,is_duplicate
0,iko industries ltd,enormous industrial trade pvt ltd,0
1,apcotex industries ltd,technocraft industries india ltd,0
2,rishichem distributors pvt ltd,dsa,0
3,powermax rubber factory,co one,0
4,tress,longyou industries park zhejiang,0
...,...,...,...
497814,bitmat products,goodyear tire rubber company,0
497815,bnd trading co ltd,zhong yue liang economy trade imp exp co ltd,0
497816,xeikon industrial co ltd dongguan city,yi cheng trading co ltd dongguan city,0
497817,shanghai kechuan trading co ltd,shanghai mg stationery inc,0


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt', max_length=12)
output = model(**encoded_input)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [5]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


1

In [6]:
cleaned_data['encoded_name_1'] = cleaned_data['name_1'].apply(lambda x: tokenizer(str(x), return_tensors='pt', padding='max_length', max_length=8)['input_ids'][0].numpy())

In [7]:
cleaned_data['encoded_name_2'] = cleaned_data['name_2'].apply(lambda x: tokenizer(str(x), return_tensors='pt', padding='max_length', max_length=8)['input_ids'][0].numpy())

In [8]:
# cleaned_data['concated'] = cleaned_data.apply(lambda row: np.concatenate((row['encoded_name_1'], row['encoded_name_2']), axis=None), axis=1)

In [9]:
dublicates = cleaned_data.loc[cleaned_data['is_duplicate']==1]

In [10]:
dublicates = dublicates.reset_index()

In [18]:
a = dublicates['encoded_name_1'][24]
b = dublicates['encoded_name_2'][24]

<h3>max lenght 30</h3>

In [19]:
1 - spatial.distance.cosine(cleaned_data['encoded_name_1'][0],cleaned_data['encoded_name_2'][0])

0.9981773672735798

In [20]:
1 - spatial.distance.cosine(a,b)

0.9987704070171306

<h3>max lenght 12</h3>

In [22]:
res_1 = spatial.distance.cosine(cleaned_data['encoded_name_1'][0],cleaned_data['encoded_name_2'][0])
res_2 = spatial.distance.cosine(a[:12],b[:12])
print(f"distance between different names: {res_1}")
print(f"distance between dublicates: {res_2}")

distance between different names: 0.2585183993433402
distance between dublicates: 0.05861742393050373


<h3>max lenght 8</h3>

In [19]:
res_1 = 1 - spatial.distance.cosine(cleaned_data['encoded_name_1'][0][:8],cleaned_data['encoded_name_2'][0][:8])
res_2 = 1 - spatial.distance.cosine(a[:8],b[:8])
print(f"distance between different names: {res_1}")
print(f"distance between dublicates: {res_2}")

distance between different names: 0.5495976604021077
distance between dublicates: 0.3321904933621317
